Onespin
===

In [26]:
import random
import math
import sys
import time

random.seed(time.time())

In [27]:
def update(s, h):
    """
    Update state by flipping spin, accept new state if E_new<E_old
    or rand[0,1)< P(new)/P(old)
    s: is the input spin
    h: is the environment (= beta H)
    """
    spin = s[0]
    newspin = -spin  # trial spin flip
    DeltaBetaE = -(newspin-spin)*h  # beta*(E(new) - E(old))
    # if the new state is at lower energy, accept it
    # if not, accept it with a probability decreasing w/ exp(-DeltaBetaE)
    if DeltaBetaE <= 0 or random.random() < math.exp(-DeltaBetaE):  # Metropolis method
        s[0] = newspin

def onespin(nsweep,h):
    print("Program generates a thermal ensemble for states with one spin.\n")
    
    # our initial state (arbitrary value, could be -1 as well)
    spin = [1]  # Using list to simulate pointer behavior
    
    # Metropolis update loop
    nplus = 0
    nminus = 0
    for n in range(nsweep):
        update(spin, h)  # try a Metropolis update
        if spin[0] == 1:
            nplus += 1
        if spin[0] == -1:
            nminus += 1
    
    print(f"P(+ state) = {nplus/(nplus+nminus):.10f}")
    print(f"P(- state) = {nminus/(nplus+nminus):.10f}")
    
    # Write <magnetization>
    print(f"<sigma> = {(nplus-nminus)/(nplus+nminus):.10f}")
    
    # Write theoretical prediction for comparison
    print(f"\nTheory prediction: sigma = tanh(h) = {math.tanh(h):.10f}")

In [28]:
onespin(100,1)
print("----")
onespin(100,1)

Program generates a thermal ensemble for states with one spin.

P(+ state) = 0.8900000000
P(- state) = 0.1100000000
<sigma> = 0.7800000000

Theory prediction: sigma = tanh(h) = 0.7615941560
----
Program generates a thermal ensemble for states with one spin.

P(+ state) = 0.8800000000
P(- state) = 0.1200000000
<sigma> = 0.7600000000

Theory prediction: sigma = tanh(h) = 0.7615941560


In [29]:
onespin(100000,0)
print("----")
onespin(100000,4)
print("----")
onespin(100000,-4)

Program generates a thermal ensemble for states with one spin.

P(+ state) = 0.5000000000
P(- state) = 0.5000000000
<sigma> = 0.0000000000

Theory prediction: sigma = tanh(h) = 0.0000000000
----
Program generates a thermal ensemble for states with one spin.

P(+ state) = 0.9996100000
P(- state) = 0.0003900000
<sigma> = 0.9992200000

Theory prediction: sigma = tanh(h) = 0.9993292997
----
Program generates a thermal ensemble for states with one spin.

P(+ state) = 0.0003900000
P(- state) = 0.9996100000
<sigma> = -0.9992200000

Theory prediction: sigma = tanh(h) = -0.9993292997


twospin
===

In [30]:
def update_spin(s, env):
    """Update spin using Metropolis algorithm"""
    spin = s[0]
    newspin = 1 if random.random() < 0.5 else -1
    DeltaBetaE = -(newspin - spin) * env
    if DeltaBetaE <= 0 or random.random() < math.exp(-DeltaBetaE):
        s[0] = newspin

def sweep(s1, s2, beta, h):
    """Sweep through both spins once"""
    spin2 = s2[0]
    update_spin(s1, beta * spin2 + h)
    spin1 = s1[0]
    update_spin(s2, beta * spin1 + h)

def twospin(nsweep,h,beta):
    print("Program generates a thermal ensemble for two coupled spins.\n")    
    
    # Initialize spins with a "hot" start
    spin1 = [-1]  # Using lists to simulate pointers
    spin2 = [-1]
    if random.random() < 0.5:
        spin1[0] = 1
    if random.random() < 0.5:
        spin2[0] = 1
    
    nupup = nupdown = ndownup = ndowndown = 0
    nmag = ncorr = ntotal = 0
    
    for _ in range(nsweep):
        sweep(spin1, spin2, beta, h)
        
        # accumulate magnetization
        nmag += spin1[0]
        ntotal += 1
        nmag += spin2[0]
        ntotal += 1
        
        # count number of times each state occurs
        if spin1[0] == 1 and spin2[0] == 1:
            nupup += 1
        elif spin1[0] == 1 and spin2[0] == -1:
            nupdown += 1
        elif spin1[0] == -1 and spin2[0] == 1:
            ndownup += 1
        elif spin1[0] == -1 and spin2[0] == -1:
            ndowndown += 1
            
        # compute spin-spin correlation function
        ncorr += spin1[0] * spin2[0]
    
    print("\nState Probabilities:")
    print(f"P(++) = {nupup/nsweep:.6f}\tP(+-) = {nupdown/nsweep:.6f}")
    print(f"P(-+) = {ndownup/nsweep:.6f}\tP(--) = {ndowndown/nsweep:.6f}")
    print(f"\nMagnetization: <s> = {nmag/(2*nsweep):.6f}")
    print("Spin-Spin Correlation Function:")
    print(f"<s1 s2> = {ncorr/nsweep:.6f}")

In [31]:
twospin(100000,0,4)

Program generates a thermal ensemble for two coupled spins.


State Probabilities:
P(++) = 0.656640	P(+-) = 0.000150
P(-+) = 0.000250	P(--) = 0.342960

Magnetization: <s> = 0.313680
Spin-Spin Correlation Function:
<s1 s2> = 0.999200


ising1d
====

In [36]:
def update_spin(s, env):
    """Do a metropolis update on a spin s whose environment is env"""
    spin = s[0]
    newspin = 1 if random.random() < 0.5 else -1
    DeltaBetaE = -(newspin - spin) * env
    if DeltaBetaE <= 0 or random.random() < math.exp(-DeltaBetaE):
        s[0] = newspin

def sweep(spin, N, beta, h):
    """Sweep once through all sites of the lattice"""
    for ns in range(1, N + 1):
        # Using lists to simulate pointers
        spin_ref = [spin[ns]]
        update_spin(spin_ref, beta * (spin[ns-1] + spin[ns+1]) + h)
        spin[ns] = spin_ref[0]

def initialize_all_spin_up(spin, N):
    """Initialize all N spins to +1 and boundary spins to 0"""
    for ns in range(1, N + 1):
        spin[ns] = 1
    spin[0] = spin[N + 1] = 0

def hot_start(spin, N):
    """Initialize spins randomly"""
    for ns in range(1, N + 1):
        spin[ns] = 1 if random.random() > 0.5 else -1
    spin[0] = spin[N + 1] = 0

def ising1d(N,nsweep,h,beta):
    ''' N = number of spins in chain
    nsweep = number of configurations (sweeps) generated
    h = value of magnetic field parameter
    beta = temperature parameter = 1/kT '''
    
    ntherm = 10000  # initial sweeps to "thermalize" the system
    VisualDisplay = 1  # 1 or 0 to turn on/off display of chains
    
    print("Program generates a thermal ensemble for chain of N spins")
    print("with free boundary conditions.\n")
    
    # make room for N spins and two fake boundary spins
    spin = [0] * (N + 2)
    
    # hot_start(spin, N)  # random spins to start
    hot_start(spin, N)
    
    # sweep ntherm times to thermalize system
    for _ in range(ntherm):
        sweep(spin, N, beta, h)
    
    # now sweep through lattice nsweep times
    nmag = ntotal = 0
    for _ in range(nsweep):
        sweep(spin, N, beta, h)
        for ns in range(1, N + 1):
            nmag += spin[ns]
            ntotal += 1
            if VisualDisplay:
                print("+", end="") if spin[ns] == 1 else print("-", end="")
        if VisualDisplay:
            print()
    
    print(f"Magnetization of thermalized system: <s> = {nmag/ntotal:.6f}")
    
    # Calculate expected result for infinite length chain
    expected = (math.exp(beta) * math.sinh(h) / 
               math.sqrt(math.exp(2*beta) * math.sinh(h) * math.sinh(h) + math.exp(-2*beta)))
    print(f"Expected result for infinite length chain over long time: {expected:.6f}")


In [37]:
ising1d(70,30,0,1.5)

Program generates a thermal ensemble for chain of N spins
with free boundary conditions.

------------------------------------++++++++++++++++++++++++++++--++++
-----------------------------------++++++++++++++++++++++++++++--+++++
------------------------------------++++++++++++++++++++++++++--------
-----------------------------------++++++++++++++++++++++++++--------+
----------------------------------+++++++++++++++++++++++++++-------++
---------------------------------++++++++++++++++++++++++++++-------++
--------------------------------++++++++++++++++++++++++++++--------++
-------------------------------++++++++++++++++++++++++++++---------++
------------------------------++++++++++++++++++++++++++++++--------++
------------------------------+++++++++++++++++++++++++++++--------+++
-----------------------------++++++++++++++++++++++++++++++-------++++
---------++++-------------------++++++++++++++++++++++++++++------++++
--------++++-------------------+++++++++++++++++++++++++++

ising2d
===

For this example use ising2.py or ising2d.cpp

ising2d_vs_T
===

For this example use ising2d_vs_T.py or ising2d_vs_T.cpp

ising1d_vs_T
===

For this example use ising1d_vs_T.cpp